### TRAIN BG

check calcul métrique   

In [21]:
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.metrics.cluster import homogeneity_score
import numpy as np

df = pd.read_csv("data.csv")

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

y = y.replace('autorisé', 0)
y = y.replace('non-autorisé', 1)

kmeans = KMeans(n_clusters=len(set(y)),init='k-means++',n_init=100, random_state=42).fit_predict(X)
kmeans

homogeneity_score(y, kmeans)

for cluster in range(len(set(y))):
  print('cluster: ', cluster)
  print(y[np.where(kmeans == cluster)])

7.61654578616956e-05

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import mlflow
from mlflow import set_tracking_uri, log_metric, log_artifact
from joblib import dump
import os

# Load the csv of embeddings
df = pd.read_csv("data.csv")

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

y = y.replace('autorisé', 0)
y = y.replace('non-autorisé', 1)

# Split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Instantiate the classification models
models = [
  LogisticRegression(), 
  SVC(probability=True), 
  DecisionTreeClassifier(), 
  RandomForestClassifier(),
  XGBClassifier(), 
  OneVsRestClassifier(SVC(kernel='linear', probability=True, verbose=True, max_iter=1000))
]

# Train the models
for model in models:
   model.fit(X_train, y_train)

# Evaluate the models on the test data
for model in models:
 y_pred = model.predict(X_test)
 y_score = model.predict_proba(X_test)[:, 1]
 sensitivity = roc_auc_score(y_test, y_score)
 specificity = 1 - sensitivity
 print(f"Score of the model {model.__class__.__name__} : {model.score(X_test, y_test)}")
 print(f"Sensitivity of the model {model.__class__.__name__} : {sensitivity}")
 print(f"Specificity of the model {model.__class__.__name__} : {specificity}")
 log_metric(f"{model.__class__.__name__}_sensitivity", sensitivity)
 log_metric(f"{model.__class__.__name__}_specificity", specificity)

# Save the results of MLflow
set_tracking_uri("http://localhost:5000")

# Create a directory for the models if it doesn't exist
if not os.path.exists("models"):
   os.makedirs("models")

# Save the model to a file and log the file as an artifact
for model in models:
   dump(model, f"models/{model.__class__.__name__}.joblib")
   log_metric("accuracy", model.score(X_test, y_test))
   log_artifact(f"models/{model.__class__.__name__}.joblib")


[LibSVM]Score of the model LogisticRegression : 0.48
Sensitivity of the model LogisticRegression : 0.49917898193760263
Specificity of the model LogisticRegression : 0.5008210180623973
Score of the model SVC : 0.48
Sensitivity of the model SVC : 0.5106732348111659
Specificity of the model SVC : 0.4893267651888341
Score of the model DecisionTreeClassifier : 0.48
Sensitivity of the model DecisionTreeClassifier : 0.4926108374384237
Specificity of the model DecisionTreeClassifier : 0.5073891625615763
Score of the model RandomForestClassifier : 0.62
Sensitivity of the model RandomForestClassifier : 0.6469622331691296
Specificity of the model RandomForestClassifier : 0.3530377668308704
Score of the model XGBClassifier : 0.5
Sensitivity of the model XGBClassifier : 0.5287356321839081
Specificity of the model XGBClassifier : 0.47126436781609193
Score of the model OneVsRestClassifier : 0.58
Sensitivity of the model OneVsRestClassifier : 0.5320197044334976
Specificity of the model OneVsRestClassi